# search csv

search all files for given header

In [18]:
import sys
sys.path.append('../../')
import glob
import shutil
from pathlib import Path
import os

import pandas as pd
from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR

from scripts.normalize_data import (
    check_duplicate_columns
)

In [19]:
clean_data_path = CLEAN_DATA_DIR

metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes.csv' 
# metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes_4.csv' 

all_columns_4_file = OUTPUT_DIR/'tmp/LIMS_4_taxa_columns_2020-02-23.csv'
all_columns_file = OUTPUT_DIR/'tmp/all_LIMS_taxa_columns_2020-02-23.csv'


## create a file with all columns  in LIMS taxa

In [37]:
metadata = pd.read_csv(metadata_file)
metadata.shape
# 1253

(1253, 16)

In [38]:
data = []
for path in metadata['path']:
    df = pd.read_csv(clean_data_path/path, dtype=str)
    df.dropna(axis="columns", how='all', inplace=True)
    for col in df.columns:
        data.append({"path": path, "column": col})

In [39]:
all_df = pd.DataFrame(data)
all_df.shape
# 60260

(60260, 2)

In [40]:
all_df.to_csv(all_columns_file)

## search for column

In [41]:
def find_column_value(all_columns_path, column, copy_files=False):
    all_df = pd.read_csv(all_columns_path, dtype=str)
    files = list(all_df[all_df['column'] == column]['path'].unique())
    
    print(f'"{column}" found in {len(files)} files\n')

    for file in files:
        path = CLEAN_DATA_DIR/file
        df = pd.read_csv(path, dtype=str, usecols=[column])
        df[column].dropna(inplace=True)
        
        print(path, '\n')
        if len(df[column].unique()) > 0:
            print(' | '.join(df[column].fillna('').unique()))
            print('----')
        
        if copy_files:
            temp_path = Path('temp', column)
            if not os.path.exists(temp_path):
                os.makedirs(temp_path)
            shutil.copy(path, temp_path)


In [42]:
column = 'Truncorotalia crassula (dextral)'

find_column_value(all_columns_file, column, False)

"Truncorotalia crassula (dextral)" found in 0 files

